# Setup

In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Path('../backend/shuscribe').resolve()
sys.path.insert(0, str(Path('../backend').resolve()))

In [2]:
# Import necessary modules
import asyncio
from dotenv import load_dotenv
import os
from shuscribe.services.llm.session import LLMSession
from shuscribe.services.llm.providers.provider import (
    Message, GenerationConfig
)
from shuscribe.schemas.llm import MessageRole

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
ANTHROPIC_API_KEY = os.environ["ANTHROPIC_API_KEY"]
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

TEST_MODELS ={
    "openai": "gpt-4o-mini",
    "anthropic": "claude-3-5-haiku-20241022",
    "gemini": "gemini-2.0-flash-001"
}

TEST_THINKING_MODELS = {
    "openai": "o3-mini-2025-01-31",
    "anthropic": "claude-3-7-sonnet-20250219",
    "gemini": "gemini-2.0-flash-thinking-exp"
}

# Helper function to run async code in notebook
async def run_async(coro):
    return await coro

In [3]:
# Streaming response
from typing import Type
from pydantic import BaseModel
from shuscribe.schemas.base import BaseOutputSchema
from shuscribe.schemas.streaming import StreamChunk
from shuscribe.services.llm.streaming import StreamStatus


async def stream(
    provider_name: str, 
    messages: list[Message], 
    response_schema: Type[BaseOutputSchema] | None = None, 
    max_tokens: int | None = None,
    temp: float | None = None
    ) -> StreamChunk | None:
    async with LLMSession.session_scope() as session:
        # Create a streaming config
        config = GenerationConfig(
            temperature=temp or 0.7,
            response_schema=response_schema if response_schema else None,
            max_output_tokens=max_tokens
        )
        
        print(f"{TEST_MODELS[provider_name]}:")

        async for chunk in session.generate_stream(
            messages=messages,
            provider_name=provider_name,
            model=TEST_MODELS[provider_name],
            config=config
        ):
            print(chunk.text, end="", flush=True)

    if chunk:
        if chunk.status == StreamStatus.COMPLETE:
            return chunk
    else:
        return None


In [4]:
import yaml

from shuscribe.schemas.pipeline import Chapter, StoryMetadata

with open("../backend/tests/resources/pokemon_amber/_meta.yaml", "r") as f:
    meta = yaml.safe_load(f)
    STORY_METADATA = StoryMetadata(
        title=meta.get('story_title'),
        description=meta.get('story_description'),
        genres=meta.get('genres'),
        additional_tags=meta.get('additional_tags')
    )

CHAPTERS = []
for chapter in meta.get('chapters'):
    with open(f"../backend/tests/resources/pokemon_amber/{chapter}", "r") as f:
        chapter_id = chapter.split('.')[0]
        try:
            chapter_content = yaml.safe_load(f)
            CHAPTERS.append(Chapter(id=chapter_id, title=chapter_content.get('title'), content=chapter_content.get('content')))
        except Exception:
            continue


# Summarization Pipeline

## Chapter Summary

In [5]:
# Set Provider Name
PROVIDER_NAME = "gemini"
CHAPTER_INDEX = 0

In [8]:
from shuscribe.schemas.wikigen.summary import ChapterSummary
from shuscribe.services.llm.prompts import templates

templates.chapter.summary.reload()
summary_messages: list[Message] = templates.chapter.summary.format( 
    current_chapter=CHAPTERS[CHAPTER_INDEX],
    story_metadata=STORY_METADATA,
    # current_chapter=Chapter(id=1, title="Chapter 1", content="This is a test chapter.")
)

# print(messages[-1].content)

summary_response = await run_async(stream(PROVIDER_NAME, summary_messages, temp=0.4))
chapter_summary = ChapterSummary.from_chapter_summary(CHAPTER_INDEX, summary_response.accumulated_text)

gemini-2.0-flash-001:
<|STARTOFSUMMARY|>

## The Fateful Raid

*   The protagonist, "Alexa," is engrossed in a Pokémon game on her phone, struggling with a difficult battle in the Pokémon Mansion.
    *   Her Gyarados is on the verge of defeat against a Vulpix.
    *   She contemplates her limited options, wary of losing another valuable Pokémon due to the game's perma-death rules.
*   A Discord notification alerts her to a Shadow Mewtwo raid event at the library gym on campus.
*   Torn between her game and the rare opportunity, Alexa saves her game state and rushes towards the library.
*   While crossing the street, distracted by her phone, Alexa is struck by a vehicle.
    *   Her last thought is about her Gyarados in the game.

## Rebirth

*   Alexa awakens in a tank filled with amber fluid, disoriented and unable to breathe properly. [!WORLD]
    *   Her senses are dulled and distorted.
    *   She experiences tremors and a deep, resonating sound.
*   The tank shatters, and she is 

Database module not implemented. Skipping save.


## Extract Entities

In [9]:
print(chapter_summary.to_prompt())

<Content>


## The Fateful Raid

*   The protagonist, "Alexa," is engrossed in a Pokémon game on her phone, struggling with a difficult battle in the Pokémon Mansion.
    *   Her Gyarados is on the verge of defeat against a Vulpix.
    *   She contemplates her limited options, wary of losing another valuable Pokémon due to the game's perma-death rules.
*   A Discord notification alerts her to a Shadow Mewtwo raid event at the library gym on campus.
*   Torn between her game and the rare opportunity, Alexa saves her game state and rushes towards the library.
*   While crossing the street, distracted by her phone, Alexa is struck by a vehicle.
    *   Her last thought is about her Gyarados in the game.

## Rebirth

*   Alexa awakens in a tank filled with amber fluid, disoriented and unable to breathe properly. [!WORLD]
    *   Her senses are dulled and distorted.
    *   She experiences tremors and a deep, resonating sound.
*   The tank shatters, and she is caught by a middle-aged man, D

In [10]:
from shuscribe.schemas.wikigen.entity import ExtractEntitiesOutSchema
print(ExtractEntitiesOutSchema.to_output_schema_str())

{
  "properties": {
    "entities": {
      "description": "important entities found in the chapter that carry narrative significance",
      "items": {
        "properties": {
          "description": {
            "description": "The description of the entity (what the entity is, what it does, etc)",
            "title": "Description",
            "type": "string"
          },
          "narrative_role": {
            "description": "The narrative role of the entity in the context of this chapter and the story as a whole",
            "title": "Narrative Role",
            "type": "string"
          },
          "significance_level": {
            "enum": [
              "Central",
              "Major",
              "Relevant",
              "Minor",
              "Incidental"
            ],
            "title": "EntitySigLvl",
            "type": "string",
            "description": "The significance level of the entity to the chapter's plot and the story as a whole"
          },


In [11]:
from shuscribe.schemas.wikigen.entity import ExtractEntitiesOutSchema


templates.entity.extract.reload()
upsert_messages: list[Message] = templates.entity.extract.format( 
    current_chapter=CHAPTERS[CHAPTER_INDEX],
    # current_chapter=Chapter(id=1, title="Chapter 1", content="This is a test chapter."),
    story_metadata=STORY_METADATA,
    chapter_summary=chapter_summary,
)

# print(extract_messages[-1].content)

extract_response = await run_async(stream(PROVIDER_NAME, upsert_messages, ExtractEntitiesOutSchema, temp=0.4))
extracted_entities = ExtractEntitiesOutSchema.model_validate_json(extract_response.accumulated_text)
print(extract_response.usage)


gemini-2.0-flash-001:
{
  "entities": [
    {
      "description": "The protagonist of the story, a Pokemon fan who gets reincarnated into the Pokemon world. At the start of the chapter, she is playing a Pokemon game on her phone.",
      "narrative_role": "The main character who experiences a sudden and unexpected isekai, setting the stage for her adventures in the Pokemon world.",
      "significance_level": "Central",
      "entity_type": "Character",
      "identifier": "Alexa",
      "aliases": [
        "AlexaTheGreat",
        "I"
      ],
      "related_entities": [
        "Gyarados",
        "Charizard",
        "Shadow Mewtwo"
      ]
    },
    {
      "description": "Alexa's level 40 Gyarados in the Pokemon game she is playing. It is a survivor of many battles and is currently close to being defeated.",
      "narrative_role": "Represents Alexa's dedication and investment in her Pokemon game. Its near-defeat triggers her decision to postpone the game and attend the Shadow 

Database module not implemented. Skipping save.


prompt_tokens=4443 completion_tokens=2003


## Upsert Entities and Relationships

In [12]:
from shuscribe.schemas.provider import LLMUsage
from shuscribe.schemas.wikigen.entity import UpsertEntitiesOutSchema
from shuscribe.schemas.wikigen.entity import EntitySigLvl

upsert_entities = UpsertEntitiesOutSchema(entities=[])
total_usage = LLMUsage(prompt_tokens=0, completion_tokens=0)
templates.entity.upsert.reload()
for batch in extracted_entities.batch_for_upsert(EntitySigLvl.RELEVANT):
    upsert_messages: list[Message] = templates.entity.upsert.format( 
        current_chapter=CHAPTERS[CHAPTER_INDEX],
        entity_batch=batch,
        story_metadata=STORY_METADATA,
        chapter_summary=chapter_summary,
    )

    upsert_response = await run_async(stream(PROVIDER_NAME, upsert_messages, UpsertEntitiesOutSchema, temp=0.4))
    upsert_entities.entities.extend(UpsertEntitiesOutSchema.model_validate_json(upsert_response.accumulated_text).entities)
    total_usage.prompt_tokens += upsert_response.usage.prompt_tokens
    total_usage.completion_tokens += upsert_response.usage.completion_tokens

print(total_usage)

gemini-2.0-flash-001:
{
  "entities": [
    {
      "old_identifier": null,
      "identifier": "Alexa",
      "detailed_description": "*   The protagonist of the story, a Pokemon fan who gets reincarnated into the Pokemon world.\n*   At the start of the chapter, she is playing a Pokemon game on her phone and struggling with a difficult battle in the Pokemon Mansion.\n*   She is hit by a truck while distracted by her phone and dies.\n*   She awakens in a tank filled with amber fluid, reborn as a child.\n*   She is identified as 'Amber' by Dr. Fuji.\n*   She experiences the world through a child's body and voice.\n*   She witnesses Mewtwo's escape and the destruction of the lab.",
      "narrative_role": "The main character who experiences a sudden and unexpected isekai, setting the stage for her adventures in the Pokemon world. She is now reborn as Amber, Dr. Fuji's daughter, and must navigate this new reality.",
      "facts": [
        {
          "fact": "Reincarnated into the body 

Database module not implemented. Skipping save.


gemini-2.0-flash-001:
{
  "entities": [
    {
      "old_identifier": null,
      "identifier": "Dr. Fuji",
      "detailed_description": "*   A middle-aged scientist who is overjoyed to see Alexa (as Amber) alive.\n*   He is the caretaker and creator of the clone body Alexa now inhabits.\n*   He calls Alexa 'Amber'.\n*   He is injured by falling debris during Mewtwo's escape, sustaining welts and a gash on his shoulder blade.\n*   He expresses a strong desire to fix their family and life.",
      "narrative_role": "The caretaker and creator of Amber, the clone body Alexa now inhabits. He is driven by a desire to restore his family and life, making him a potentially complex and morally ambiguous figure.",
      "facts": [
        {
          "fact": "He is injured by falling debris during Mewtwo's escape.",
          "type": "Explicit"
        },
        {
          "fact": "He expresses a strong desire to fix their family and life.",
          "type": "Explicit"
        }
      ],
   

Database module not implemented. Skipping save.


prompt_tokens=10709 completion_tokens=3715


In [13]:
print(upsert_entities.model_dump_json(indent=2))

{
  "entities": [
    {
      "old_identifier": null,
      "identifier": "Alexa",
      "detailed_description": "*   The protagonist of the story, a Pokemon fan who gets reincarnated into the Pokemon world.\n*   At the start of the chapter, she is playing a Pokemon game on her phone and struggling with a difficult battle in the Pokemon Mansion.\n*   She is hit by a truck while distracted by her phone and dies.\n*   She awakens in a tank filled with amber fluid, reborn as a child.\n*   She is identified as 'Amber' by Dr. Fuji.\n*   She experiences the world through a child's body and voice.\n*   She witnesses Mewtwo's escape and the destruction of the lab.",
      "narrative_role": "The main character who experiences a sudden and unexpected isekai, setting the stage for her adventures in the Pokemon world. She is now reborn as Amber, Dr. Fuji's daughter, and must navigate this new reality.",
      "facts": [
        {
          "fact": "Reincarnated into the body of Amber, Dr. Fuji's d

## Story So Far Summary

In [16]:

templates.story.comprehensive_summary.reload()
comprehensive_summary_messages: list[Message] = templates.story.comprehensive_summary.format( 
    current_chapter=CHAPTERS[CHAPTER_INDEX],
    chapter_summary=chapter_summary,
    key_entities=upsert_entities,
    
    story_metadata=STORY_METADATA,
    # summary_so_far=summary_so_far,
    # recent_summaries=recent_summaries_prompt,
)

# print(comprehensive_summary_messages[-1].content)

comprehensive_summary_response = await run_async(stream(PROVIDER_NAME, comprehensive_summary_messages, temp=0.4))
# comprehensive_summary = ChapterSummary.from_chapter_summary(CHAPTER_INDEX, comprehensive_summary_response.accumulated_text)


gemini-2.0-flash-001:
<|STARTOFSUMMARY|>

## Introduction

"Pokemon: Ambertwo" is a fan fiction that follows a Pokemon enthusiast named [[Alexa]] who is unexpectedly reincarnated into the Pokemon world. She finds herself in the body of [[Amber]], the daughter/clone of [[Dr. Fuji]], a scientist known for his research into genetics and the creation of [[Mewtwo]]. The story explores themes of identity, family, and the ethical implications of scientific advancement within a world that blends elements from various Pokemon media.

## Major Plot Arcs

### Reincarnation and Identity

[[Alexa]], a dedicated Pokemon gamer, meets an untimely end when she is struck by a vehicle while distracted by a [[Shadow Mewtwo Raid]]. She awakens to find herself reborn as [[Amber]], a young girl in a tank filled with amber fluid inside [[Dr. Fuji's Lab]]. This sets the stage for her journey of adapting to a new identity and navigating the complexities of the Pokemon world from an entirely different perspectiv

Database module not implemented. Skipping save.
